## Atari Games

**Atari 2600** is a video game console which provides several popular games such as pong, space invaders, pacman, break out, centipede and many more. Gymnasium includes [ALE-py](https://pypi.org/project/ale-py/), the **Arcade Learning Environment (ALE)** which is a simple framework that allows researchers to develop AI agents for Atari 2600 games. It is built on top of the **Stella emulator** and separates the details of emulation from agent design. The main Gymnasium page provides a list of all the [Atari environments](https://gymnasium.farama.org/environments/atari/). The objective of this notebook is to **train an agent to play a game of the Atari console** using the DQN algorithm **learning from raw pixel data**, like in the famous paper [V. Mnih, et al. **"Playing Atari with Deep Reinforcement Learning"**, NIPS 2013](./../papers/2013%20-%20Playing%20Atari%20with%20Deep%20Reinforcement%20Learning.pdf):

![](./images/atari.png)

In particular, we can consider **the classic [Pong](https://gymnasium.farama.org/environments/atari/pong/) game**. The game is played like tennis. Using a controller, each player hits the ball by moving vertically the paddles on the game field. A player scores one point when the opponent misses a hit. The first player who scores 21 points wins the game. The action space is discrete and deterministic with 6 actions (not all meaningful for Pong): 0 (noop), 1 (fire), 2 (right) and 3 (left), 4 (right fire), 5 (left fire) The observation space is the screen image (210x160x3). The state space is really high-dimensional and to use it you need to modify the neural network architecture adding some convolutional layer in order to extract features from the image.

1 - Define a variable with the name of the Atari game you want to solve (e.g. Pong) and create an instance of the environment.

In [ ]:
# YOUR CODE HERE

# Import Gymnasium and create an instance with "render_mode" set to "rgb_array". You 
# need to import also "ale_py" plugin and to register it with Gymnasium (gym.register_envs(ale_py)).
# Set also the parameter "frameskip" to 1. Later, we will see that we want to control
# the number of frames to skip as a parameter of our code.


A.L.E: Arcade Learning Environment (version 0.10.1+6a7e0ae)
[Powered by Stella]


2 - Create a function to show the game in action by rendering the environment and selecting actions at random or using a policy.

In [ ]:
# YOUR CODE HERE

# Define a function "random_pi" that takes as input a state and a Q-function and 
# returns a random action.


In [ ]:
# YOUR CODE HERE

# Define a function "show_policy" that takes as input an environment, 
# a policy and a Q-function and shows the policy in action for a number of episodes


3 - Use the previous function in order to see one episode of the game with a random policy.

In [1]:
# YOUR CODE HERE

# Just call the "show_policy" function defined above passing the environment, None as agent and 
# the number of episodes you want to visualize


4 - We would like to use as state the raw pixels data. However, **directly feeding the raw game screen image is not efficient**, since it will be **computationally expensive**. So we need to pre-process the environment output to reduce the dimensionality of the problem. Moreover the Atari environments have some game specific issues, which are not relevant for the learning process, that we need to manage. In order to do that, you can use the **wrapper function**, which is a convenient way to modify an existing environment without having to alter the underlying code directly. Using wrappers will allow you to avoid a lot of boilerplate code and make our environment more modular. Here a list of the useful modifications that we can apply to the Atari environment: 

1) The raw game screen size is 210x160 pixels with three color channels (RGB), however we don't need the color information and we can resize the image in order to reduce the dimensionality of the problem.

![](./images/resize.png)

2) Atari games typically run at 60 frames per second, however not every frame contains crucial information for decision-making. To reduce computation and memory requirements, and to speed up training, we can skip a certain number of frames. Moreover, some games have a flickering effect (when the game draws different portions of the screen on even and odd frames) due to the old Atari platform’s limitation. For the human eye, such quick changes are not visible, but they can confuse a neural network. So, we can take the maximum of every pixel in the last two frames and use it as observation.

3) In order to inject variability into the initial state of each episode, promoting exploration and helping the agent discover a more robust policy across different starting conditions, we can introduce a certain number of "no-operation" actions at the beginning of each episode in order to add stochasticity to the initial conditions of the environment. 

Add a wrapper in order to apply all those modifications to the environment.

In [2]:
# YOUR CODE HERE

# Gymnasium provides a wrapper to apply those pre-porcessing to the 
# observations for the Atari games, you can use the "AtariPreprocessing" wrapper 
# from "gymnasium.wrappers" module.


5 - A single raw screenshot of the game used as state is not enough to extract the information about the game state. How can the agent learn to play just seeing a single image at a time? It is **not possible to build an idea about how the game objects are moving**. In Pong, just looking at a single frame don't provide enough information to understand if the ball is moving left to right or right to left. For example, in the Cart-pole example, the state is composed by the position of the cart, the angle of the pole, but it includes also the velocity of both. To deal with this, we need to consider as state a series of frame and not just one. In this way, the agent can learn how the objects are moving. So, we need to apply a wrapper that will stack the last N (e.g. 4) frames together and use them as state.

![](./images/stack.png)

In [3]:
# YOUR CODE HERE

# Gymnasium provides the "FrameStackObservation" wrapper to stack the last n frames of the game. Apply it to the
# environment.



6 - Some games, as Pong, require a user to press the FIRE button to start the game, we need to take this action on reset. Add another wrapper to take this action on reset.


In [4]:
# YOUR CODE HERE

# This wrapper che be found in th stable_baselines3 library. It is called "FireResetEnv"
# Import it and apply it to the environment.


7 - Finally, we write a "make_atari_env()" function that takes as input the name of the environment and returns the wrapped environment

![](./images/pre-processing.png)

In [5]:
# YOUR CODE HERE

# You create an environment and then apply in order all the wrappers needed


8 - Since we are dealing with images, **instead of using a fully connected neural network, this time we need to use a convolutional neural network**. The convolutional layers extract the features from the image and output the feature maps, then the flattened feature map can be fed to the fully connected layer which returns the Q value. You can implement the following architecture:

- Convolutional layer with 32 filters, kernel size 8, stride 4, padding 0, followed by a ReLU activation function
- Convolutional layer with 64 filters, kernel size 4, stride 2, padding 0, followed by a ReLU activation function
- Convolutional layer with 64 filters, kernel size 3, stride 1, padding 0, followed by a ReLU activation function
- Flatten layer
- Fully connected layer with 512 units, followed by a ReLU activation function
- Fully connected layer with a number of units equal to the number of actions 

![](./images/network.png)

In [6]:
# YOUR CODE HERE

# Add the code for th neural network using the PyTorch library. And 
# create two networks, one for the online network and one for the target network.  


9 - Add the Replay Memory to store the transitions (state, action, reward, next_state, done).

In [7]:
# YOUR CODE HERE

# Add the code for the replay buffer already used presented in the DQN implementation.
#  We need a larger replay buffer because the problem is more complex. 


10 - Create a the code for exploration (decaying epsilon-greedy policy) and exploitation (greedy policy) and the code for doing the evaluation of the agent.

In [8]:
# YOUR CODE HERE

# Add the code for decaying epsilon and the epsilon-greedy policy as already
# done for the DQN implementation


In [9]:
# YOUR CODE HERE

# Add the code for an evaluation or the agent as already
# done for the DQN implementation.


11 - Add the code for the optimization of the Q-function based on the experience sampled from the replay buffer. Remember to use the target network to compute the target Q-values.

In [10]:
# YOUR CODE HERE

# Add the code of the optimization algorithm as already
# done for the DQN implementation. This time you need to to pay attention
# to states which are a stack of four images


12 - Write the code of the DQN algorithm (exploiting the new convolutional network architecture). You should modify the code in order to save the network weights after some episodes.

In [11]:
# YOUR CODE HERE

# Add the code for the DQN algorithm, exploiting the convolutional neural architecture
# already defined, the replay buffer as already implemented in DQN algorithm. 
# At the begging, if a model file is already present on the disk, you should load the weights and 
# sometimes, you should save the model on the disk.


14 - Apply the algorithm to the ATARI environment. Pay attention, the following code takes a lot of time to run (days on a powerful machine).

In [12]:
# YOUR CODE HERE

# Apply the previous written algorithm to the ATARI environment, please pay attention
# to the parameters.


15 - Smooth the result using a sliding window

In [13]:
# YOUR CODE HERE

# smooth the result using a sliding window
        

16 - Show the score curve as a function of the number of episodes

In [14]:
# YOUR CODE HERE

# Show the moving average reward during the training process


18 - Show the agent playing the game using the greedy policy   

In [15]:
# YOUR CODE HERE

# You can show the policy learned by the agent using the "show_policy" function
